In [6]:
from esc_to_tfrecords import *
from esc_reader import*

##### Create tfrecords

In [2]:
FS = 16000
SPLIT = 4
AUGMENT = 9
STRONG = True

esc_path = '/workspace/esc/esc10/'

npz_path = '{}wav{}.npz'.format(esc_path, FS//1000)
tfrecord_pathes = ['{}wav{}_train.tfrecord'.format(esc_path, FS//1000),
                   '{}wav{}_val.tfrecord'.format(esc_path, FS//1000)]


create_tfrecords(npz_path, tfrecord_pathes,
                 split = SPLIT, fs = FS,
                 augment_factor = AUGMENT, strong = STRONG)

Preprocessing sounds...
Augmenting data...
Preprocessing sounds...
Augmenting data...
Preprocessing sounds...
Augmenting data...
Preprocessing sounds...
Augmenting data...
Preprocessing sounds...
Augmenting data...
2870 2870
740 740
Writing tfrecords...


#####  Read tfrecords without defining a graph
Create generator to iterate through tfrecords 

In [4]:
tfrecord_path = tfrecord_pathes[0]
record_iterator  =  tf.python_io.tf_record_iterator(path=tfrecord_path)

for string_record in record_iterator:
    example = tf.train.Example()
    example.ParseFromString(string_record)
    
    
    label = example.features.feature['label'].int64_list.value[0]
    speaker = example.features.feature['speaker'].int64_list.value[0]
    sr = example.features.feature['sr'].int64_list.value[0]

    signal_string = example.features.feature['signal_raw'].bytes_list.value[0]
    signal = np.frombuffer(signal_string, dtype = np.uint8)
    
    print(label, speaker, sr)
    print(signal.shape)
    break

7 7 16000
(160000,)


##### Create dataset

In [7]:
dataset = ESCDataset(tfrecord_path=tfrecord_path)

##### Get dataset batch

In [8]:
# get a batch in the following format: tf.Example protobuf parsed from tfrecord
batch = dataset.get_example(batch_size = 10)
print(batch.keys())

batch

dict_keys(['label', 'signal_raw', 'speaker', 'sr'])


{'label': <tf.Tensor 'ParseSingleExample/ParseSingleExample:0' shape=() dtype=int64>,
 'signal_raw': <tf.Tensor 'ParseSingleExample/ParseSingleExample:1' shape=() dtype=string>,
 'speaker': <tf.Tensor 'ParseSingleExample/ParseSingleExample:2' shape=() dtype=int64>,
 'sr': <tf.Tensor 'ParseSingleExample/ParseSingleExample:3' shape=() dtype=int64>}

In [9]:
#get a batch in the following format: {key:tensor} 
batch = dataset.get_wavenet_batch(batch_size = 10)
print(batch.keys())

batch

dict_keys(['wav', 'sr', 'speaker', 'label'])


{'wav': <tf.Tensor 'Reshape_1:0' shape=(10, 40000) dtype=float32>,
 'sr': <tf.Tensor 'strided_slice:0' shape=(10,) dtype=int32>,
 'speaker': <tf.Tensor 'strided_slice_1:0' shape=(10,) dtype=int32>,
 'label': <tf.Tensor 'strided_slice_2:0' shape=(10,) dtype=int32>}

##### Define graph to read tfrecords and  iterate through batches

In [11]:
dataset = ESCDataset(tfrecord_path=tfrecord_path)

LENGTH = 40000
batch = dataset.get_wavenet_batch(batch_size = 50, length = LENGTH) 

# The op for initializing the variables.
init_op = tf.group(tf.global_variables_initializer(),
                   tf.local_variables_initializer())

with tf.Session() as sess:
    
    sess.run(init_op)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord = coord)
    
    for i in range(3):
    
        batch_np = sess.run(batch)
        features, labels = batch_np['wav'], batch_np['label']

        print(features.shape, labels.shape)
        
    coord.request_stop()
    coord.join(threads)

(50, 40000) (50,)
(50, 40000) (50,)
(50, 40000) (50,)
